# Mantid->MCViNE conversion
## Create working directory for the conversion and move over there

In [ ]:
# modify this
workdir = "/SNS/users/lj7/simulations/mantid2mcvine_instrument_conversion/demo"
!mkdir -p {workdir}
%cd {workdir}

## Add the package into the python path

In [ ]:
import sys
sys.path.insert(0, '/SNS/users/lj7/simulations/mantid2mcvine_instrument_conversion')

In [ ]:
import os
import mantid2mcvine as m2m
reload(m2m)

## Instrument information

In [ ]:
# modify these
instrument_name = 'MCVINEDEMO'
beamline = 101
mantid_idf = os.path.abspath('../notebooks/MCVINEDEMO_Definition.xml')
mcvine_idf = os.path.abspath('mcvine.xml')
template_nxs = os.path.abspath('template.nxs')

detsys_shape = m2m.shapes.hollowCylinder(in_radius=2., out_radius=3., height=3.) # meters
nbanks = 1
ntubesperpack = 8
npixelspertube = 128
nmonitors = 1
    
tube_info = m2m.TubeInfo(
    pressure = 10.*m2m.units.pressure.atm,
    radius = .5 * m2m.units.length.inch,
    gap = 0.08 * m2m.units.length.inch,
)

tofbinsize = 0.1 # mus

## Create instrument model
This model will be used for the converting Mantid IDF to MCViNE IDF below,
and later for converting MCViNE-simulated neutrons to nexus file understood 
by Mantid.

In [ ]:
im = m2m.InstrumentModel(
    instrument_name, beamline, mantid_idf, mcvine_idf, template_nxs,
    detsys_shape, tube_info,
    nbanks = nbanks,
    ntubesperpack = ntubesperpack,
    npixelspertube = npixelspertube,
    nmonitors = nmonitors,
    tofbinsize = tofbinsize,
    )

## Run IDF conversion
The IDF conversion only needs to be done once!!!

In [ ]:
im.convert()
im.mantid_install() # install mantid IDF to ~/.mantid

## Save instrument model for future use

In [ ]:
import pickle as pkl
# save
pkl.dump(im, open('instrument_model.pkl', 'w'))

# load
# im2 = pkl.load(open('instrument_model.pkl'))

# Test generated files by running a powder simulation

In [ ]:
import numpy as np, h5py
import histogram as H, histogram.hdf as hh
from matplotlib import pyplot as plt
import mcvine
%matplotlib inline

In [ ]:
#!mcvine

In [ ]:
pwd

In [ ]:
powder_workdir = "/SNS/users/lj7/simulations/mantid2mcvine_instrument_conversion/demo/powder-cones"
!mkdir -p {powder_workdir}
%cd {powder_workdir}

In [ ]:
# rm -rf mysim/

In [ ]:
!mcvine workflow powder --instrument=ARCS --sample=V --workdir=mysim

## Sim Beam

In [ ]:
cd {powder_workdir}/mysim/beam

In [ ]:
%%file run-beam.sh
#!/usr/bin/env bash

mcvine instruments arcs beam -E=70 --ncount=1e7 --nodes=20

In [ ]:
%%time
%%script bash --out beam_out --err beam_err
./run-beam.sh

In [ ]:
# print beam_out
# print beam_err

In [ ]:
# plot I(E) spectrum
ie = hh.load("out/ienergy.h5")
plt.plot(ie.energy, ie.I)

## Scattering sim

In [ ]:
cd {powder_workdir}/mysim

### Change sample kernel

In [ ]:
%%file sampleassembly/V-scatterer.xml
<?xml version="1.0"?>

<!DOCTYPE scatterer>

<!-- mcweights: monte-carlo weights for 3 possible processes:
     absorption, scattering, transmission -->
<homogeneous_scatterer mcweights="0, 1, 0.1">

  <KernelContainer average="yes">

    <Phonon_IncoherentElastic_Kernel dw_core='0.00701434948808*angstrom**2'>
    </Phonon_IncoherentElastic_Kernel>

    <ConstantQEKernel momentum-transfer="4/angstrom" energy-transfer="10*meV"/>

    <ConstantQEKernel momentum-transfer="7/angstrom" energy-transfer="20*meV"/>

    <ConstantQEKernel momentum-transfer="10/angstrom" energy-transfer="10*meV"/>

    </KernelContainer>
</homogeneous_scatterer>

### Scattering script

In [ ]:
%%file scatter
#!/usr/bin/env bash

LAUNCHER=`python -c "from mcni.pyre_support.MpiApplication import mpi_launcher_choice as l; print l"`
rm -rf out && ./sss --ncount=3e6 --multiple-scattering=off --$LAUNCHER.nodes=20 --buffer_size=50000

In [ ]:
!chmod +x scatter sss reduce2iqe create-nxs

### Simulate

In [ ]:
!make clean

In [ ]:
%%time
%%script bash --out scatter_out --err scatter_err
./scatter

In [ ]:
# print scatter_err

In [ ]:
ls -lt out/scattered-neutrons

## Create NeXus file

In [ ]:
%cd {powder_workdir}/mysim

In [ ]:
%%time
events = im.neutrons2events('./out/scattered-neutrons', nodes=20)

In [ ]:
ls -lt n2e/out/events.dat

In [ ]:
%%time
simnxs = im.events2nxs(events, 'sim.nxs')

## Reduction

In [ ]:
from mantid import simpleapi as msa
reload(msa)

In [ ]:
cd {powder_workdir}/mysim

In [ ]:
ws = msa.Load('sim.nxs')

In [ ]:
out = msa.DgsReduction(
    SampleInputWorkspace=ws, 
    IncidentEnergyGuess=70., UseIncidentEnergyGuess=1,
    TimeZeroGuess=20)

In [ ]:
msa.SofQW3(
    InputWorkspace='out',
    OutputWorkspace='iqw',
    QAxisBinning="0,0.1,12",
    EMode='Direct',
    )

In [ ]:
msa.SaveNexus(
    InputWorkspace='iqw',
    Filename = 'iqe.nxs',
    Title = 'iqw',
    )

In [ ]:
!mcvine mantid extract_iqe iqe.nxs iqe.h5

In [ ]:
iqe = hh.load("iqe.h5").transpose()
H.plot(iqe, min=0, max=.003)